# Fuzzy ARTMAP com dados embaralhados

Esse modelo é formado por duas módulos de FuzzyART.

O aprendizado feito é supervisionado.

Aqui, embaralharemos os dados e avaliaremos o modelo usando a métrica ACC.

<font color="red">Dúvida:</font>
É possível usar a average accuracy mesmo com os dados embaralhados?

## Importando bibliotecas

In [2]:
from torchvision import datasets, transforms
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from artlib import FuzzyARTMAP
from sklearn.metrics import accuracy_score
import pandas as pd

## Baixando base de dados

In [3]:
transform = transforms.ToTensor()
dataset = datasets.USPS(root='./data', train=True, download=True, transform=transform)

X = np.stack([img.numpy() for img, _ in dataset])
y = np.array([label for _, label in dataset])

# Achatar as imagens
X = X.reshape(X.shape[0], -1)

X, y = shuffle(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print(X.shape, y.shape)

(7291, 256) (7291,)


## Treinando o modelo

In [4]:
alphas = [0.0, 1e-1, 1.0] 
rhos   = [0.1, 0.5, 0.9]
betas  = [1.0]

results = []

for alpha in alphas:
    for rho in rhos:
        for beta in betas:

            model = FuzzyARTMAP(rho=rho, alpha=alpha, beta=beta)
            
            lower_bound, upper_bound = (model.module_a).find_data_bounds(X)
            model.module_a.set_data_bounds(lower_bound, upper_bound)

            # Preparar os dados (normalizados internamente pelo modelo)
            train_X_prep = model.prepare_data(X_train)
            test_X_prep  = model.prepare_data(X_test)

            # Treinar
            model.fit(train_X_prep, y_train)

            # Prever
            predictions = model.predict(test_X_prep)

            # Calcular accuracy
            acc = accuracy_score(y_test, predictions)

            results.append({
                'alpha': alpha,
                'rho': rho,
                'beta': beta,
                'accuracy': acc
            })


df_results = pd.DataFrame(results)
worst_results = df_results.sort_values(by='accuracy', ascending=True).head(10)
best_results = df_results.sort_values(by='accuracy', ascending=True).tail(10)


In [6]:
print("10 melhores combinações:")
print(best_results)

print("10 piores combinações:")
print(worst_results)

10 melhores combinações:
   alpha  rho  beta  accuracy
0    0.0  0.1   1.0  0.787796
6    1.0  0.1   1.0  0.869729
3    0.1  0.1   1.0  0.873157
7    1.0  0.5   1.0  0.940007
4    0.1  0.5   1.0  0.940692
1    0.0  0.5   1.0  0.942407
2    0.0  0.9   1.0  0.958176
5    0.1  0.9   1.0  0.958176
8    1.0  0.9   1.0  0.958176
10 piores combinações:
   alpha  rho  beta  accuracy
0    0.0  0.1   1.0  0.787796
6    1.0  0.1   1.0  0.869729
3    0.1  0.1   1.0  0.873157
7    1.0  0.5   1.0  0.940007
4    0.1  0.5   1.0  0.940692
1    0.0  0.5   1.0  0.942407
2    0.0  0.9   1.0  0.958176
5    0.1  0.9   1.0  0.958176
8    1.0  0.9   1.0  0.958176
